In [1]:
from datetime import datetime

import torch
from config import *

from data.compress import *
from data.util import count_points_between, crop_q_between

%reload_ext autoreload
%autoreload 2

In [2]:
READ_START_DATE = datetime.strptime(CONFIG['READ_START_DATE'], DATE_FORMAT)
READ_END_DATE = datetime.strptime(CONFIG['READ_END_DATE'], DATE_FORMAT)
TRAIN_START_DATE = datetime.strptime(CONFIG['TRAIN_START_DATE'], DATE_FORMAT)
TRAIN_END_DATE = datetime.strptime(CONFIG['TRAIN_END_DATE'], DATE_FORMAT)
ALPHA = CONFIG['ALPHA']

Load matrix $Q$

In [3]:
mat_q = torch.load(out_path('mat_q_resid.pt'))
mat_q = torch.abs(mat_q)
mat_q.shape

torch.Size([32064, 472])

Crop $Q$ to the required time period

In [4]:
n_samples, n_sections = mat_q.shape
mat_q = crop_q_between(mat_q, READ_START_DATE, READ_END_DATE, TRAIN_START_DATE, TRAIN_END_DATE)
assert mat_q.shape == (count_points_between(TRAIN_START_DATE, TRAIN_END_DATE), n_sections)
mat_q.shape

torch.Size([2880, 472])

Construct a correlation coefficient matrix
$$
R(i, j)=\frac{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))(z(s_j,t_k)-\tilde{z}(s_j))}{\sqrt{\sum_{k=1}^d(z(s_i,t_k)-\tilde{z}(s_i))^2}\sqrt{\sum_{k=1}^d(z(s_j,t_k)-\tilde{z}(s_j))^2}},
$$
where $$\tilde{z}(s_i)=\frac{1}{d}\sum_{k=1}^dz(s_i,t_k)$$

In [5]:
mat_r = build_correlation_matrix(mat_q)
mat_r.shape

torch.Size([472, 472])

In [6]:
groups, n_ungrouped = split_sections_into_groups(mat_r, ALPHA)

Analyse grouping

In [7]:
_, n_sections = mat_q.shape
print(f'Using alpha={ALPHA}, {n_sections} correlated sections were divided ' +
      f'into {len(groups)} groups:')
n_ungrouped = 0
for i, (group, corr_min, corr_max) in enumerate(groups, start=1):
      group_coeffs = mat_r[tuple(group.T), :]
      print(f'Group {i} - {len(group)} sections with correlation coefficients '
          f'{corr_min:.3f} to {corr_max:.3f}')
print(str(n_ungrouped) + " section(s) don't correlate with anything and weren't grouped")


Using alpha=0.8, 472 correlated sections were divided into 12 groups:
Group 1 - 6 sections with correlation coefficients 0.844 to 1.000
Group 2 - 29 sections with correlation coefficients 0.629 to 0.781
Group 3 - 61 sections with correlation coefficients 0.500 to 0.624
Group 4 - 73 sections with correlation coefficients 0.401 to 0.498
Group 5 - 46 sections with correlation coefficients 0.317 to 0.395
Group 6 - 32 sections with correlation coefficients 0.252 to 0.315
Group 7 - 27 sections with correlation coefficients 0.192 to 0.240
Group 8 - 12 sections with correlation coefficients 0.150 to 0.187
Group 9 - 9 sections with correlation coefficients 0.106 to 0.129
Group 10 - 11 sections with correlation coefficients 0.069 to 0.085
Group 11 - 2 sections with correlation coefficients 0.063 to 0.063
Group 12 - 9 sections with correlation coefficients 0.027 to 0.033
0 section(s) don't correlate with anything and weren't grouped


/tmp/ipykernel_19875/3401722382.py:6: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3571.)
  group_coeffs = mat_r[tuple(group.T), :]


In [8]:
mat_c = get_compression_matrix(mat_q, groups)

assert mat_c.shape == (mat_q.shape[0], len(groups))
torch.save(mat_c, out_path('mat_c.pt'))
mat_c.shape

torch.Size([2880, 12])

In [9]:
mat_x = get_compressed_matrix(mat_c, mat_q)
torch.save(mat_x, out_path('mat_x.pt'))
mat_x.shape

torch.Size([12, 472])